# Not Scaled Data

In [9]:
import pandas as pd
xls = pd.ExcelFile("Фрустрация.xlsx")
data = pd.read_excel(xls, xls.sheet_names)    

In [10]:
keys = [' E', ' E1', ' e_', ' M', ' M1', ' m_', ' I', ' I1', ' i_']

keys1 = [[' E', ' E1', ' e_'],
         [' M', ' M1', ' m_'],
         [' I', ' I1', ' i_']]

keys2 = [[' E', ' M', ' I'],
        [' E1', ' M1', ' I1'],
        [' e_', ' m_', ' i_']]
def get_y(keys, data, init_keys = keys):
    y = []
    for ind, i in enumerate(init_keys):
        for ind1, j in enumerate(keys):
            if i in j:
                y.append(ind1 * np.ones(len(data[i])))
    return np.hstack(y)

In [11]:
data[keys[0]].columns

Index(['Unnamed: 0', 'Число восклицательных знаков',
       'Число вопросительных знаков', 'Число слов', 'Число символов',
       'Число уникальных слов', 'Число клауз', 'Число предложений',
       'Число знаков препинания',
       'Средняя длина предложения (в количестве слов)',
       ...
       'Синтакс. связь: nummod', 'Синтакс. связь: nummod:gov',
       'Синтакс. связь: obj', 'Синтакс. связь: obl', 'Синтакс. связь: orphan',
       'Синтакс. связь: parataxis', 'Синтакс. связь: punct',
       'Синтакс. связь: root', 'Синтакс. связь: xcomp',
       'Синтакс. связь: compound'],
      dtype='object', length=243)

In [49]:
def extract(list_of_texts):
    res = []
    for s in list_of_texts:
        k = 0
        for c in s:
            if c.islower():
                k += 1
        res.append(k/len(s))
    
    return pd.DataFrame({"Доля":res})

In [53]:
import json
with open("p3_texts.json", "r") as f:
    texts = f.read()
    texts = json.loads(texts)
    f.close()
print(texts.keys())
for key in keys:
    if key[-1] == '1':
        key1 = key[:-1] + "'"
    else:
        key1 = key
    data[key] = data[key].join( extract(texts['P3:' + key1]), rsuffix = "___")

dict_keys(['P3: E', 'P3: M', 'P3: m_', 'P3: e_', "P3: I'", "P3: E'", 'P3: i_', "P3: M'", 'P3: инф', 'P3: I', 'P3: n ("надо")'])


In [54]:
import numpy as np
from sklearn.preprocessing import scale
X = np.vstack([data[i].values for i in keys])
X = X[:, 1:]
y1 = get_y(keys1, data)
y2 = get_y(keys2, data)

In [55]:
print("Число слов", X[:, 2].mean())

Число слов 36.49063670411985


In [56]:
print("Число предложений", X[:, 6].mean())

Число предложений 3.4569288389513106


In [57]:
print("Число символов", X[:, 3].mean())

Число символов 188.49812734082397


In [68]:
X = scale(X)
X_bin = np.where(X <= 0, 0, 1)

## The First Union

### 1. Not binarized data

In [71]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.utils import shuffle

X_, y = shuffle(X, y1)
print(X_.shape)
model = LogisticRegression(penalty = "l1", solver = "liblinear", class_weight = "balanced", multi_class = "ovr")
cross_val_score(model, X_, y, cv = 3, scoring = "f1_macro").mean()

(267, 244)


0.40771426058987853

In [72]:
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA

X_ = PCA(1 - 1e-3).fit(X_).transform(X_)
print(X_.shape)
model = LogisticRegression(penalty = "l1", solver = "liblinear", class_weight = "balanced", multi_class = "ovr")
cross_val_score(model, X_, y, cv = 3, scoring = "f1_macro").mean()

(267, 157)


0.3588310610957485

In [73]:
from sklearn.ensemble import GradientBoostingClassifier

model = GradientBoostingClassifier(n_estimators = 1000, max_depth = 10)

cross_val_score(model, X_, y, cv = 3, scoring = "f1_macro").mean()

/home/elias/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


0.2902479880897147

### 2. Binarized data

In [74]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.utils import shuffle

X_, y = shuffle(X_bin, y1)
print(X_.shape)
model = LogisticRegression(penalty = "l1", solver = "liblinear", class_weight = "balanced", multi_class = "ovr")
cross_val_score(model, X_, y, cv = 3, scoring = "f1_macro").mean()

(267, 244)


0.379303937888361

In [75]:
from sklearn.linear_model import LogisticRegression

X_ = PCA(1 - 1e-3).fit(X_).transform(X_)
print(X_.shape)
model = LogisticRegression(penalty = "l1", solver = "liblinear", class_weight = "balanced", multi_class = "ovr")
cross_val_score(model, X_, y, cv = 3, scoring = "f1_macro").mean()

(267, 157)


0.38142900787486106

In [76]:
from sklearn.ensemble import GradientBoostingClassifier

model = GradientBoostingClassifier(n_estimators = 1000, max_depth = 10)

cross_val_score(model, X_, y, cv = 3, scoring = "f1_macro").mean()

0.27696624752994825

## The Second Union

### 1. Not binarized data

In [77]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.utils import shuffle

X_, y = shuffle(X, y2)
print(X_.shape)
model = LogisticRegression(penalty = "l1", solver = "liblinear", class_weight = "balanced", multi_class = "ovr")
cross_val_score(model, X_, y, cv = 3, scoring = "f1_macro").mean()

(267, 244)


0.3347149968677063

In [78]:
from sklearn.linear_model import LogisticRegression

X_ = PCA(1 - 1e-3).fit(X_).transform(X_)
print(X_.shape)
model = LogisticRegression(penalty = "l1", solver = "liblinear", class_weight = "balanced", multi_class = "ovr")
cross_val_score(model, X_, y, cv = 3, scoring = "f1_macro").mean()

(267, 157)


0.3195803893180063

In [79]:
from sklearn.ensemble import GradientBoostingClassifier

model = GradientBoostingClassifier(n_estimators = 10, max_depth = 5)

cross_val_score(model, X_, y, cv = 3, scoring = "f1_macro").mean()

0.275569203514438

### 2. Binarized data

In [81]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.utils import shuffle

X_, y = shuffle(X_bin, y2)
print(X_.shape)
model = LogisticRegression(penalty = "l1", solver = "liblinear", class_weight = "balanced", multi_class = "ovr")
cross_val_score(model, X_, y, cv = 3, scoring = "f1_macro").mean()

(267, 244)


0.36633540613435606

In [82]:
from sklearn.linear_model import LogisticRegression

X_ = PCA(1 - 1e-3).fit(X_).transform(X_)
print(X_.shape)
model = LogisticRegression(penalty = "l1", solver = "liblinear", class_weight = "balanced", multi_class = "ovr")
cross_val_score(model, X_, y, cv = 3, scoring = "f1_macro").mean()

(267, 157)


0.3736874634724097

In [83]:
from sklearn.ensemble import GradientBoostingClassifier

model = GradientBoostingClassifier(n_estimators = 1000, max_depth = 10)

cross_val_score(model, X_, y, cv = 3, scoring = "f1_macro").mean()

0.3700205855600216